In [1]:
import h5py
import scipy.io as sio
import numpy as np
import pandas as pd


Ce notebook a pour but d'extraire les data matrices dont nous avons besoin pour travailler. Il faut commencer par sélectionner la discrétisation voulue. 

A l'issu, nous avons un nouveau .h5 SC_FC_dataset_filtered_discretisationxdiscretisation.h5 et un csv avec le nom des région corticale. A noter que le filtre ixc est appliqué.

In [7]:
discretisation_possible = {83 : 0, 129 : 1, 234 : 2, 463 : 3, 1015 : 4} 
discretisation_chosen = discretisation_possible[83]

In [8]:
# Charger le masque cortical
labels_data = sio.loadmat('Data/labels_index_CORTICAL_Laus2008_all_scales.mat')
ixc = labels_data['ixc'][0, discretisation_chosen][0] - 1  # Ajustement d'indexation

# Chemin du fichier de données
file_path = 'Data/27_SCHZ_CTRL_dataset.mat'
output_path = f'Data_ML/SC_FC_dataset_filtered_{len(ixc)}x{len(ixc)}.h5'

with h5py.File(file_path, 'r') as f:
    with h5py.File(output_path, 'w') as hf:
        for key in f['SC_FC_Connectomes'].keys():
            if key == 'demographics':  # Ignorer demographics
                continue
            group = f['SC_FC_Connectomes'][key]
            hf.create_group(key)
            
            for sub_key in group.keys():
                data = group[sub_key][discretisation_chosen, 0]  # Référence
                matrix = f[data][:]  # Charger la matrice
                filtered_matrix = matrix[:, ixc, :][:, :, ixc]  # Appliquer le filtre
                hf[key].create_dataset(sub_key, data=filtered_matrix)

output_path


'Data_ML/SC_FC_dataset_filtered_68x68.h5'

In [9]:
# Ouvrir le fichier HDF5
file_path = output_path

with h5py.File(file_path, 'r') as hf:
    def explore_group(group, indent=0):
        for key in group.keys():
            item = group[key]
            if isinstance(item, h5py.Group):
                print("  " * indent + f"📂 Groupe: {key}")
                explore_group(item, indent + 1)
            else:
                print("  " * indent + f"🗂️ Dataset: {key}, shape={item.shape}, dtype={item.dtype}")

    print("Clés principales :", list(hf.keys()))
    explore_group(hf)


Clés principales : ['FC_correlation', 'SC_ADC', 'SC_density', 'SC_gFA', 'SC_length_of_fibers', 'SC_number_of_fibers']
📂 Groupe: FC_correlation
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float32
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float32
📂 Groupe: SC_ADC
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float64
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float64
📂 Groupe: SC_density
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float64
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float64
📂 Groupe: SC_gFA
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float64
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float64
📂 Groupe: SC_length_of_fibers
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float64
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float64
📂 Groupe: SC_number_of_fibers
  🗂️ Dataset: ctrl, shape=(27, 68, 68), dtype=float64
  🗂️ Dataset: schz, shape=(27, 68, 68), dtype=float64


In [10]:
# Charger le fichier de labels
labels_file = 'Data/labels_index_CORTICAL_Laus2008_all_scales.mat'
labels_data = sio.loadmat(labels_file)

# Extraire le masque cortical et la liste des labels
llist = labels_data['llist']

# Créer une liste des noms des régions corticales
region_names = [llist[0, discretisation_chosen][i, 0][0] for i in range(llist[0, discretisation_chosen].shape[0])]

# Créer un DataFrame
df_regions = pd.DataFrame({'Indice': range(1, len(region_names) + 1), 'Région Cérébrale': region_names})

# Enregistrer au format CSV
csv_path = f'Data_ML/regions_corticales_{llist[0, discretisation_chosen].shape[0]}_indices.csv'
df_regions.to_csv(csv_path, index=False)

csv_path


'Data_ML/regions_corticales_68_indices.csv'